# Zero-Coupon Bond Using Marlowe Runtime\'s Command-Line Interface

***Before running this notebook, you might want to use Jupyter's "clear output" function to erase the results of the previous execution of this notebook. That will make more apparent what has been executed in the current session.***

The zero-coupon bond example is a simple Marlowe contract where a lender provides principal to a borrower who repays it back with interest.

In this demonsration we use Marlowe Runtime\'s command-line interface, `marlowe-runtime-cli`, to run this contract on Cardano\'s `preprod` public testnet. Marlowe contracts may use either addresses or role tokens for authorization: here we use addresses.

[A video works through this Jupyter notebook.](https://youtu.be/pjDtuD5rimI)

You can ask questions about Marlowe in [the #ask-marlowe channel on the IOG Discord](https://discord.com/channels/826816523368005654/936295815926927390) or post problems with this lesson to [the issues list for the Marlowe Starter Kit github repository](https://github.com/input-output-hk/marlowe-starter-kit/issues).

In [Marlowe Playground](https://play.marlowe.iohk.io//), the contract looks like this in Blockly format.

![Zero-coupon bond Marlowe contract](images/01-zcb-contract.png)

In Marlowe format it appears as
```
When
    [Case
        (Deposit
            (Address "$LENDER_ADDR")
            (Address "$LENDER_ADDR")
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
        )
        (Pay
            (Address "$LENDER_ADDR")
            (Party (Address "$BORROWER_ADDR"))
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
            (When
                [Case
                    (Deposit
                        (Address "$BORROWER_ADDR")
                        (Address "$BORROWER_ADDR")
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                    )
                    (Pay
                        (Address "$BORROWER_ADDR")
                        (Party (Address "$LENDER_ADDR"))
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                        Close 
                    )]
                (TimeParam "$BORROWER_DEADLINE")
                Close 
            )
        )]
    (TimeParam "$LENDER_DEADLINE")
    Close 
```

## Preliminaries

See [Lesson 0. Preliminaries](00-preliminaries.md) for information on setting up one's environment for using this tutorial.

The lesson assumes that the following environment variables have been set.
- `CARDANO_NODE_SOCKET_PATH`: location of Cardano node's socket.
- `CARDANO_TESTNET_MAGIC`: testnet magic number.
- `MARLOWE_RT_HOST`: IP address of the Marlowe Runtime proxy server.
- `MARLOWE_RT_PORT`: Port number for the Marlowe Runtime proxy server.

It also assumes that the Lender and Borrower parties have addresses, signing keys, and funds.
- Lender
    - [keys/lender.address](keys/lender.address): Cardano address for the lender
    - [keys/lender.skey](keys/lender.skey): location of signing key file for the lender
- Borrower
    - [keys/borrower.address](keys/borrower.address): Cardano address for the borrower
    - [keys/borrower.skey](keys/borrower.skey): location of signing key file for the borrower

### Access to Cardano node and Marlowe Runtime

If we're using [demeter.run](https://demeter.run/)'s Cardano Marlowe Runtime extension, then we already have access to Cardano Node and Marlowe Runtime. The followind commands will set the required environment variables to use a local docker deployment on the default ports. It will also set some supplementary environment variables.

In [ ]:
if [[ -z "$MARLOWE_RT_PORT" ]]
then

  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$(docker volume inspect marlowe-starter-kit_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=1 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

  # Only required for `marlowe-runtime-cli`.
  export MARLOWE_RT_HOST="127.0.0.1"
  export MARLOWE_RT_PORT=3700

fi

# FIXME: This should have been inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should have been set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

case "$CARDANO_TESTNET_MAGIC" in
  1)
    export "EXPLORER_URL=https://preprod.cardanoscan.io"
    ;;
  2)
    export "EXPLORER_URL=https://preview.cardanoscan.io"
    ;;
  *)
    # Use `mainnet` as the default.
    export "EXPLORER_URL=https://cardanoscan.io"
    ;;
esac

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"
echo "MARLOWE_RT_HOST = $MARLOWE_RT_HOST"
echo "MARLOWE_RT_PORT = $MARLOWE_RT_PORT"

Note the test network magic number:
- `preprod` = 1
- `preview` = 2

### Lender address and funds

Check that an address and key has been created for the lender. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [ ]:
LENDER_SKEY=keys/lender.skey
LENDER_ADDR=$(cat keys/lender.address)
echo "LENDER_ADDR = $LENDER_ADDR"

Check that the lender has at least one hundred ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/address/"$LENDER_ADDR"

### Borrower address and funds

Check that an address and key has been created for the borrower. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [ ]:
BORROWER_SKEY=keys/borrower.skey
BORROWER_ADDR=$(cat keys/borrower.address)
echo "BORROWER_ADDR = $BORROWER_ADDR"

Check that the borrower has at least one hundred ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/address/"$BORROWER_ADDR"

## Design the contract

The zero-coupon bond contract can be downloaded from the [Marlowe Playground](https://play.marlowe.iohk.io//) as a JSON file, or it can be generated using [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) using the `marlowe-cli template` command.

Here we generate the contract using Marlowe CLI.

First, set the loan's principal to 80 ada and its interest to 5 ada.

In [ ]:
ADA=1000000  # 1 ada = 1,000,000 lovelace

PRINCIPAL=$((80 * ADA))
INTEREST=$((5 * ADA))

echo "PRINCIPAL = $PRINCIPAL lovelace"
echo "INTEREST = $INTEREST lovelace"

On the Cardano blockchain, the protocol parameters require that each UTxO contain at least some ada. Here we will start the contract with 2 ada.

In [ ]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

Next find the current time, measured in [POSIX milliseconds](https://en.wikipedia.org/wiki/Unix_time).

In [ ]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

NOW="$((`date -u +%s` * SECOND))"
echo NOW = "$NOW" POSIX milliseconds = "`date -d @$((NOW / SECOND))`"

The contract has a lending deadline and a repayment deadline. For convenience in this example, set the deadlines to the near future.

In [ ]:
LENDER_DEADLINE="$((NOW + 1 * HOUR))"
BORROWER_DEADLINE="$((NOW + 3 * HOUR))"
echo LENDER_DEADLINE = "$LENDER_DEADLINE" POSIX milliseconds = "`date -d @$((LENDER_DEADLINE / SECOND))`"
echo BORROWER_DEADLINE = "$BORROWER_DEADLINE" POSIX milliseconds = "`date -d @$((BORROWER_DEADLINE / SECOND))`"

Now create the JSON file for the contract, `zcb-contract.json`.

In [ ]:
marlowe-cli template zcb \
  --minimum-ada "$MIN_LOVELACE" \
  --lender "$LENDER_ADDR" \
  --borrower "$BORROWER_ADDR" \
  --principal "$PRINCIPAL" \
  --interest "$INTEREST" \
  --lending-deadline "$LENDER_DEADLINE" \
  --repayment-deadline "$BORROWER_DEADLINE" \
  --out-contract-file zcb-contract.json \
  --out-state-file zcb-state.json

The various command-line options are described by the help system.

In [ ]:
marlowe-cli template zcb --help

## Examine the contract

View the contract file as YAML.

In [ ]:
json2yaml zcb-contract.json

Also view the initial state of the contract. Note that Marlowe Runtime overrides this with a state that it builds.

In [ ]:
json2yaml zcb-state.json

### \[Optional, but recommended\] Check the safety of the contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe.iohk.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://play.marlowe.iohk.io//) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

Here we\'ll perform step 6. First we bundle the contract and its initial state into a single file.

In [ ]:
marlowe-cli run initialize \
  --permanently-without-staking \
  --contract-file zcb-contract.json \
  --state-file zcb-state.json \
  --out-file zcb-marlowe.json

Now we analyze the contract and its execution paths.

In [ ]:
marlowe-cli run analyze \
  --marlowe-file zcb-marlowe.json

In the above report, we see that the contract doesn\'t have any duplicate or invalid values, and it does not exceed any of the blockchain\'s protocol parameters. In particular, note that our previously chosen `MIN_LOVELACE` value of 2 ada is greater than the 1.120600 ada that the analysis tool says is needed. Thus, it is safe to execute any path in the contract.

## Transaction 1. Create the Contract

Marlowe Runtime\'s command `marlowe-runtime-cli create` will build the creation transaction for a Marlowe contract. We provide it the JSON file containing the contract and tell it the `MIN_LOVELACE` value that we previously chose. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

In [ ]:
marlowe-runtime-cli create --help

In [ ]:
CONTRACT_ID=$(
marlowe-runtime-cli create \
  --core-file zcb-contract.json \
  --min-utxo "$MIN_LOVELACE" \
  --change-address "$LENDER_ADDR" \
  --manual-sign tx-1.unsigned \
| jq -r 'fromjson | .contractId' \
)
echo "CONTRACT_ID = $CONTRACT_ID"

Marlowe Runtime uses the first (creation) UTxO of the contract to identify it throughout its lifecycle.

The result of building the transaction is the identifier for the contract and the file `tx-1.unsigned`, which contains the Cardano unsigned transaction for creating the contract, in text-envelope format.

In [ ]:
json2yaml tx-1.unsigned

There are many ways to sign and submit Cardano transactions:
- `cardano-cli` at the command line
- `cardano-wallet` at the command line or as a REST service
- `cardano-hw-cli` for a hardware wallet at the command line
- a Babbage-compatible CIP-30 wallet in a web browser
- `marlowe-cli` at the command line
- `marlowe-runtime-cli submit` at the command line (requires signing by a separate tool such as `cardano-cli`)

For convenience, here we use `marlowe-cli transaction submit`. One may have to wait a minute or so for the transactions to be confirmed on the blockchain.

In [ ]:
TX_1=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_1?tab=utxo"

One can also examine the contract's UTxO using `cardano-cli`.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_1#1"

## View the details of the contract on the blockchain

Marlowe Runtime\'s command `marlowe-runtime-cli log` can fetch a contract from the blockchain and print information about it.

In [ ]:
marlowe-runtime-cli log --show-contract "$CONTRACT_ID"

More detail can be retrieved using `marlowe-pipe`.

In [ ]:
echo '{"request" : "get", "contractId" : "'"$CONTRACT_ID"'"}' | marlowe-pipe 2> /dev/null | json2yaml

## Transaction 2. The lender deposits the principal

The lender deposits their 80 ada of principal into the contract using Marlowe Runtime\'s `marlowe-runtime-cli deposit` command. The lender is providing the funding for and receiving the change from this transaction, so we provide their address. We provide the contract identifier and save the unsigned transaction in the file `tx-2.unsigned`.

In [ ]:
marlowe-runtime-cli deposit --help

In [ ]:
TX_2=$(
marlowe-runtime-cli deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$LENDER_ADDR" \
  --to-party "$LENDER_ADDR" \
  --lovelace "$PRINCIPAL" \
  --change-address "$LENDER_ADDR" \
  --manual-sign tx-2.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_2 = $TX_2"

Note that if the transaction would violate the logic of the Marlowe contract, one would receive an error message. For example, let\'s say that we deposit the incorrect amount or deposit it to the wrong party\'s internal account.

In [ ]:
marlowe-runtime-cli deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$LENDER_ADDR" \
  --to-party "$LENDER_ADDR" \
  --lovelace 80 \
  --change-address "$LENDER_ADDR" \
  --manual-sign /dev/null

In [ ]:
marlowe-runtime-cli deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$LENDER_ADDR" \
  --to-party "$BORROWER_ADDR" \
  --lovelace "$PRINCIPAL"\
  --change-address "$LENDER_ADDR" \
  --manual-sign /dev/null

The [Marlowe Debugging Cookbook](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/debugging-cookbook.md) guides interpretation of error messages. Also, one can determine the possible actions for the contract at its current stage of execution by studing the contract\'s current state or by using Marlowe playground to simulate the contract.

![Simulation of zero-coupon bond contract in Marlowe Playground](images/zcb-simulation.png)

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600s

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_2?tab=utxo"

One can see that the lender has 82 ada less than originally. Two ada were deposited in the contract when it was created and 80 ada were paid to the borrower in the second transaction.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

The borrower has an additional 80 ada (the loan's principal) now.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

The Marlowe contract still has the 2 ada from its creation.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

## View the further progress of the contract on the blockchain

Marlowe Runtime\'s command `marlowe-runtime-cli log` can fetch a contract from the blockchain and print information about it.

In [ ]:
marlowe-runtime-cli log --show-contract "$CONTRACT_ID"

More detail can be retrieved using `marlowe-pipe`.

In [ ]:
echo '{"request" : "get", "contractId" : "'"$CONTRACT_ID"'"}' | marlowe-pipe 2> /dev/null | json2yaml

## Transaction 3. The borrower repays the loan

After some time passes, the borrower repays principal plus interest. Thus, they fund the transaction and receive the change at their address.

In [ ]:
TX_3=$(
marlowe-runtime-cli deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$BORROWER_ADDR" \
  --to-party "$BORROWER_ADDR" \
  --lovelace "$((PRINCIPAL+INTEREST))" \
  --change-address "$BORROWER_ADDR" \
  --manual-sign tx-3.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_3 = $TX_3"

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$BORROWER_SKEY" \
  --timeout 600s

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_3?tab=utxo"

One can see that the lender received back the 80 ada of principal and the 2 ada deposited when the contract was created, along with the additional 5 ada of interest, totallying 87 ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

The borrower now has about 5 ada (the loan's interest) less than originally.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

The Marlowe contract has closed, so there is no output to its script address.

## View the completion of the contract on the blockchain

Marlowe Runtime\'s command `marlowe-runtime-cli log` can fetch a contract from the blockchain and print information about it.

In [ ]:
marlowe-runtime-cli log --show-contract "$CONTRACT_ID"

More detail can be retrieved using `marlowe-pipe`.

In [ ]:
echo '{"request" : "get", "contractId" : "'"$CONTRACT_ID"'"}' | marlowe-pipe 2> /dev/null | json2yaml